<a href="https://colab.research.google.com/github/GabrielData21/ETL-com-PySpark/blob/main/ETL_com_Python_e_PySpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pipeline de ETL com Python (PySpark)

**Descrição da atividade**

Esse projeto visa simular uma pipeline contendo importação e transformação de dados do ENEM em uma arquitetura medallion, cujo perfil consiste em 3 camadas de tratamento de dados, sendo uma camada bruta no momento da importação (bronze), uma segunda camada para pré-processamento (silver) e a camada final com os dados prontos para uso (gold).


## Instalando PySpark

Realizando a instalação do PySpark no notebook. Em versões anteriores era necessário instalar o ambiente com Hadoop e outros módulos, a partir da versão 3.4.0 é possível fazer a instação isolada.

In [1]:
# instalando PySpark 3.4.0
!pip install pyspark==3.4.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317122 sha256=47722dd1b731e34de2d41da5b87ad0d439d58a8612d226d64fe826c9a83e9407
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [2]:
# criando Spark Session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("ETL-enem").getOrCreate()
spark

# Organizando o ambiente para os arquivos do ETL

Criando as pastas que irão compor o projeto de acordo com a arquitetura medallion.

In [3]:
!mkdir -p enem_2020/1.bronze

In [4]:
!mkdir enem_2020/2.silver/ enem_2020/3.gold/

## Aquisição de dados ENEM

### Download e pré processamento dos dados

In [5]:
# Importando os arquivos
!wget https://download.inep.gov.br/microdados/microdados_enem_2020.zip --no-check-certificate

# verificando os arquivos na pasta após download
!ls -l

--2024-05-15 13:34:01--  https://download.inep.gov.br/microdados/microdados_enem_2020.zip
Resolving download.inep.gov.br (download.inep.gov.br)... 200.130.24.15
Connecting to download.inep.gov.br (download.inep.gov.br)|200.130.24.15|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: 620776982 (592M) [application/zip]
Saving to: ‘microdados_enem_2020.zip’

microdados_enem_202 100%[===================>] 592.02M   721KB/s    in 14m 37s 

2024-05-15 13:48:40 (691 KB/s) - ‘microdados_enem_2020.zip’ saved [620776982/620776982]

total 606240
drwxr-xr-x 5 root root      4096 May 15 13:33 enem_2020
-rw-r--r-- 1 root root 620776982 Feb 16  2022 microdados_enem_2020.zip
drwxr-xr-x 1 root root      4096 May 14 20:30 sample_data


Os arquivos do ENEM são recebidos em formato compactado, para fazer a descompactação existem duas opções: usando comandos linux shell (através do unzip) ou usando o pacote zipfile do Python. Como este projeto visa uma prática em Python e PySpark, foi usado o pacote zipfile.

In [6]:
import zipfile

In [7]:
%%time

# realizando a descompactação dos arquivos
with zipfile.ZipFile('microdados_enem_2020.zip', 'r') as zip_ref:
    zip_ref.extractall('enem_2020')

CPU times: user 13.1 s, sys: 6.22 s, total: 19.3 s
Wall time: 24.1 s


Outro passo necessário é organizar os arquivos da descompactação, será usado apenas o arquivo principal para a transformação. Por isso ele será movido para a pasta /bronze.

Mover arquivos entre pastas usando comandos linux shell é bem simples através do comando mv. Novamente, para manter o treinamento voltado para Python, usaremos o pacote shutil para realizar a movimentação.

In [8]:
import shutil

In [10]:
# criando variáveis para movimentação de arquivo
source = "/content/enem_2020/DADOS/MICRODADOS_ENEM_2020.csv"
destination = "/content/enem_2020/1.bronze/MICRODADOS_ENEM_2020.csv"

# movimentando o arquivo
shutil.move(source, destination)

'/content/enem_2020/1.bronze/MICRODADOS_ENEM_2020.csv'

In [31]:
%%time

# importando pacote Functions para manipulação de dados
import pyspark.sql.functions

# primeira leitura dos dados descompactados
mida = spark.read.csv('/content/enem_2020/1.bronze/MICRODADOS_ENEM_2020.csv', header= True, sep=';', encoding='ISO-8859-1')
mida.limit(10)

CPU times: user 4.82 ms, sys: 0 ns, total: 4.82 ms
Wall time: 372 ms


DataFrame[NU_INSCRICAO: string, NU_ANO: string, TP_FAIXA_ETARIA: string, TP_SEXO: string, TP_ESTADO_CIVIL: string, TP_COR_RACA: string, TP_NACIONALIDADE: string, TP_ST_CONCLUSAO: string, TP_ANO_CONCLUIU: string, TP_ESCOLA: string, TP_ENSINO: string, IN_TREINEIRO: string, CO_MUNICIPIO_ESC: string, NO_MUNICIPIO_ESC: string, CO_UF_ESC: string, SG_UF_ESC: string, TP_DEPENDENCIA_ADM_ESC: string, TP_LOCALIZACAO_ESC: string, TP_SIT_FUNC_ESC: string, CO_MUNICIPIO_PROVA: string, NO_MUNICIPIO_PROVA: string, CO_UF_PROVA: string, SG_UF_PROVA: string, TP_PRESENCA_CN: string, TP_PRESENCA_CH: string, TP_PRESENCA_LC: string, TP_PRESENCA_MT: string, CO_PROVA_CN: string, CO_PROVA_CH: string, CO_PROVA_LC: string, CO_PROVA_MT: string, NU_NOTA_CN: string, NU_NOTA_CH: string, NU_NOTA_LC: string, NU_NOTA_MT: string, TX_RESPOSTAS_CN: string, TX_RESPOSTAS_CH: string, TX_RESPOSTAS_LC: string, TX_RESPOSTAS_MT: string, TP_LINGUA: string, TX_GABARITO_CN: string, TX_GABARITO_CH: string, TX_GABARITO_LC: string, TX_G

In [13]:
"""
Verificando as colunas do DataFrame, avaliando junto à documetação do dicionário dos dados
sobre o conteúdo, formato...
"""
mida.printSchema()

root
 |-- NU_INSCRICAO: string (nullable = true)
 |-- NU_ANO: string (nullable = true)
 |-- TP_FAIXA_ETARIA: string (nullable = true)
 |-- TP_SEXO: string (nullable = true)
 |-- TP_ESTADO_CIVIL: string (nullable = true)
 |-- TP_COR_RACA: string (nullable = true)
 |-- TP_NACIONALIDADE: string (nullable = true)
 |-- TP_ST_CONCLUSAO: string (nullable = true)
 |-- TP_ANO_CONCLUIU: string (nullable = true)
 |-- TP_ESCOLA: string (nullable = true)
 |-- TP_ENSINO: string (nullable = true)
 |-- IN_TREINEIRO: string (nullable = true)
 |-- CO_MUNICIPIO_ESC: string (nullable = true)
 |-- NO_MUNICIPIO_ESC: string (nullable = true)
 |-- CO_UF_ESC: string (nullable = true)
 |-- SG_UF_ESC: string (nullable = true)
 |-- TP_DEPENDENCIA_ADM_ESC: string (nullable = true)
 |-- TP_LOCALIZACAO_ESC: string (nullable = true)
 |-- TP_SIT_FUNC_ESC: string (nullable = true)
 |-- CO_MUNICIPIO_PROVA: string (nullable = true)
 |-- NO_MUNICIPIO_PROVA: string (nullable = true)
 |-- CO_UF_PROVA: string (nullable = tru

# Transformação dos dados

In [15]:
"""
Para realizar o tratamento de forma mais eficiente, o arquivo será transformado em Parquet.
É possível atribuir o destino do arquivo em uma variável para usar na função
de salvar o arquivo no formato desejado, por isso ao alterar o formato o arquivo será salvo na pasta /silver.
"""

# Caminho para salvar o arquivo Parquet
parquet_path = "/content/enem_2020/2.silver/MICRODADOS_ENEM_2020.parquet"

# Salvar o DataFrame como arquivo Parquet
mida.write.mode("overwrite").parquet(parquet_path)

In [32]:
%%time

# Pra ler o arquivo Parquet, pode usar a mesma variável criada para alocar o aquivo na célula anterior
mida_parquet = spark.read.parquet(parquet_path)
mida_parquet.limit(10)

CPU times: user 6.02 ms, sys: 0 ns, total: 6.02 ms
Wall time: 149 ms


DataFrame[NU_INSCRICAO: string, NU_ANO: string, TP_FAIXA_ETARIA: string, TP_SEXO: string, TP_ESTADO_CIVIL: string, TP_COR_RACA: string, TP_NACIONALIDADE: string, TP_ST_CONCLUSAO: string, TP_ANO_CONCLUIU: string, TP_ESCOLA: string, TP_ENSINO: string, IN_TREINEIRO: string, CO_MUNICIPIO_ESC: string, NO_MUNICIPIO_ESC: string, CO_UF_ESC: string, SG_UF_ESC: string, TP_DEPENDENCIA_ADM_ESC: string, TP_LOCALIZACAO_ESC: string, TP_SIT_FUNC_ESC: string, CO_MUNICIPIO_PROVA: string, NO_MUNICIPIO_PROVA: string, CO_UF_PROVA: string, SG_UF_PROVA: string, TP_PRESENCA_CN: string, TP_PRESENCA_CH: string, TP_PRESENCA_LC: string, TP_PRESENCA_MT: string, CO_PROVA_CN: string, CO_PROVA_CH: string, CO_PROVA_LC: string, CO_PROVA_MT: string, NU_NOTA_CN: string, NU_NOTA_CH: string, NU_NOTA_LC: string, NU_NOTA_MT: string, TX_RESPOSTAS_CN: string, TX_RESPOSTAS_CH: string, TX_RESPOSTAS_LC: string, TX_RESPOSTAS_MT: string, TP_LINGUA: string, TX_GABARITO_CN: string, TX_GABARITO_CH: string, TX_GABARITO_LC: string, TX_G

Parte do processo de ETL é a seleção das colunas que contém os dados importantes para análises posteriores pela equipe de Analytics.

Em um dataset que contém muitas colunas como esse, para manter o código limpo é interessante colocar as colunas selecionadas para deleção em uma variável antes de colocar na função. Isso é realizado utilizando o documento de dicionário de dados.

In [17]:
# Imprimindo as colunas em uma lista
list(mida_parquet.columns)

['NU_INSCRICAO',
 'NU_ANO',
 'TP_FAIXA_ETARIA',
 'TP_SEXO',
 'TP_ESTADO_CIVIL',
 'TP_COR_RACA',
 'TP_NACIONALIDADE',
 'TP_ST_CONCLUSAO',
 'TP_ANO_CONCLUIU',
 'TP_ESCOLA',
 'TP_ENSINO',
 'IN_TREINEIRO',
 'CO_MUNICIPIO_ESC',
 'NO_MUNICIPIO_ESC',
 'CO_UF_ESC',
 'SG_UF_ESC',
 'TP_DEPENDENCIA_ADM_ESC',
 'TP_LOCALIZACAO_ESC',
 'TP_SIT_FUNC_ESC',
 'CO_MUNICIPIO_PROVA',
 'NO_MUNICIPIO_PROVA',
 'CO_UF_PROVA',
 'SG_UF_PROVA',
 'TP_PRESENCA_CN',
 'TP_PRESENCA_CH',
 'TP_PRESENCA_LC',
 'TP_PRESENCA_MT',
 'CO_PROVA_CN',
 'CO_PROVA_CH',
 'CO_PROVA_LC',
 'CO_PROVA_MT',
 'NU_NOTA_CN',
 'NU_NOTA_CH',
 'NU_NOTA_LC',
 'NU_NOTA_MT',
 'TX_RESPOSTAS_CN',
 'TX_RESPOSTAS_CH',
 'TX_RESPOSTAS_LC',
 'TX_RESPOSTAS_MT',
 'TP_LINGUA',
 'TX_GABARITO_CN',
 'TX_GABARITO_CH',
 'TX_GABARITO_LC',
 'TX_GABARITO_MT',
 'TP_STATUS_REDACAO',
 'NU_NOTA_COMP1',
 'NU_NOTA_COMP2',
 'NU_NOTA_COMP3',
 'NU_NOTA_COMP4',
 'NU_NOTA_COMP5',
 'NU_NOTA_REDACAO',
 'Q001',
 'Q002',
 'Q003',
 'Q004',
 'Q005',
 'Q006',
 'Q007',
 'Q008',
 'Q009

In [18]:
# Selecionando as colunas que serão removidas
dropcols = ('TX_RESPOSTAS_CN',
 'TX_RESPOSTAS_CH',
 'TX_RESPOSTAS_LC',
 'TX_RESPOSTAS_MT',
 'TX_GABARITO_CN',
 'TX_GABARITO_CH',
 'TX_GABARITO_LC',
 'TX_GABARITO_MT',
 'Q001',
 'Q002',
 'Q003',
 'Q004',
 'Q005',
 'Q006',
 'Q007',
 'Q008',
 'Q009',
 'Q010',
 'Q011',
 'Q012',
 'Q013',
 'Q014',
 'Q015',
 'Q016',
 'Q017',
 'Q018',
 'Q019',
 'Q020',
 'Q021',
 'Q022',
 'Q023',
 'Q024',
 'Q025')

In [35]:
mida_parquet = mida_parquet.drop(*dropcols)
mida_parquet.limit(10)

DataFrame[NU_INSCRICAO: string, NU_ANO: string, TP_FAIXA_ETARIA: string, TP_SEXO: string, TP_ESTADO_CIVIL: string, TP_COR_RACA: string, TP_NACIONALIDADE: string, TP_ST_CONCLUSAO: string, TP_ANO_CONCLUIU: string, TP_ESCOLA: string, TP_ENSINO: string, IN_TREINEIRO: string, CO_MUNICIPIO_ESC: string, NO_MUNICIPIO_ESC: string, CO_UF_ESC: string, SG_UF_ESC: string, TP_DEPENDENCIA_ADM_ESC: string, TP_LOCALIZACAO_ESC: string, TP_SIT_FUNC_ESC: string, CO_MUNICIPIO_PROVA: string, NO_MUNICIPIO_PROVA: string, CO_UF_PROVA: string, SG_UF_PROVA: string, TP_PRESENCA_CN: string, TP_PRESENCA_CH: string, TP_PRESENCA_LC: string, TP_PRESENCA_MT: string, CO_PROVA_CN: string, CO_PROVA_CH: string, CO_PROVA_LC: string, CO_PROVA_MT: string, NU_NOTA_CN: string, NU_NOTA_CH: string, NU_NOTA_LC: string, NU_NOTA_MT: string, TP_LINGUA: string, TP_STATUS_REDACAO: string, NU_NOTA_COMP1: string, NU_NOTA_COMP2: string, NU_NOTA_COMP3: string, NU_NOTA_COMP4: string, NU_NOTA_COMP5: string, NU_NOTA_REDACAO: string]

In [20]:
mida_parquet.count()

5783109

Até esse momento foi realizado uma transformação simples dos dados alterando o formato do arquivo e removendo colunas não essenciais para o trabalho de analytics. Agora o arquivo se encontra na pasta /silver.

In [21]:
# Listando as colunas
mida_parquet.columns

['NU_INSCRICAO',
 'NU_ANO',
 'TP_FAIXA_ETARIA',
 'TP_SEXO',
 'TP_ESTADO_CIVIL',
 'TP_COR_RACA',
 'TP_NACIONALIDADE',
 'TP_ST_CONCLUSAO',
 'TP_ANO_CONCLUIU',
 'TP_ESCOLA',
 'TP_ENSINO',
 'IN_TREINEIRO',
 'CO_MUNICIPIO_ESC',
 'NO_MUNICIPIO_ESC',
 'CO_UF_ESC',
 'SG_UF_ESC',
 'TP_DEPENDENCIA_ADM_ESC',
 'TP_LOCALIZACAO_ESC',
 'TP_SIT_FUNC_ESC',
 'CO_MUNICIPIO_PROVA',
 'NO_MUNICIPIO_PROVA',
 'CO_UF_PROVA',
 'SG_UF_PROVA',
 'TP_PRESENCA_CN',
 'TP_PRESENCA_CH',
 'TP_PRESENCA_LC',
 'TP_PRESENCA_MT',
 'CO_PROVA_CN',
 'CO_PROVA_CH',
 'CO_PROVA_LC',
 'CO_PROVA_MT',
 'NU_NOTA_CN',
 'NU_NOTA_CH',
 'NU_NOTA_LC',
 'NU_NOTA_MT',
 'TP_LINGUA',
 'TP_STATUS_REDACAO',
 'NU_NOTA_COMP1',
 'NU_NOTA_COMP2',
 'NU_NOTA_COMP3',
 'NU_NOTA_COMP4',
 'NU_NOTA_COMP5',
 'NU_NOTA_REDACAO']

Para a área de Analytics, as colunas selecionadas serão renomeadas.

In [22]:
enem_gold = mida_parquet.withColumnsRenamed({
 'NU_INSCRICAO':'INSCRICAO',
 'NU_ANO':'ANO',
 'TP_FAIXA_ETARIA':'FAIXA_ETARIA',
 'TP_SEXO':'SEXO',
 'TP_COR_RACA':'COR_RACA',
 'TP_ST_CONCLUSAO':'ST_CONCLUSAO',
 'TP_ANO_CONCLUIU':'ANO_CONCLUIU',
 'TP_ESCOLA':'ESCOLA',
 'TP_ENSINO':'ENSINO',
 'IN_TREINEIRO':'TREINEIRO',
 'SG_UF_PROVA':'UF_PROVA',
 'TP_PRESENCA_CN':'PRE_C_NATURAIS',
 'TP_PRESENCA_CH':'PRE_C_HUMANAS',
 'TP_PRESENCA_LC':'PRE_LINGUAGENS',
 'TP_PRESENCA_MT':'PRE_MATEMATICA',
 'NU_NOTA_CN':'NOTA_C_NATURAIS',
 'NU_NOTA_CH':'NOTA_C_HUMANAS',
 'NU_NOTA_LC':'NOTA_LINGAGENS',
 'NU_NOTA_MT':'NOTA_MATEMATICA',
 'TP_LINGUA':'LINGUA_ESTR',
 'TP_STATUS_REDACAO':'STATUS_REDACAO',
 'NU_NOTA_COMP1':'NOTA_COMP1',
 'NU_NOTA_COMP2':'NOTA_COMP2',
 'NU_NOTA_COMP3':'NOTA_COMP3',
 'NU_NOTA_COMP4':'NOTA_COMP4',
 'NU_NOTA_COMP5':'NOTA_COMP5',
 'NU_NOTA_REDACAO':'NOTA_REDACAO'
 })

In [34]:
# Visualizando os dados com as colunas renomeadas
enem_gold.limit(5)

DataFrame[INSCRICAO: string, ANO: string, FAIXA_ETARIA: string, SEXO: string, COR_RACA: string, ST_CONCLUSAO: string, ANO_CONCLUIU: string, ESCOLA: string, ENSINO: string, TREINEIRO: string, UF_PROVA: string, PRE_C_NATURAIS: string, PRE_C_HUMANAS: string, PRE_LINGUAGENS: string, PRE_MATEMATICA: string, NOTA_C_NATURAIS: int, NOTA_C_HUMANAS: int, NOTA_LINGAGENS: int, NOTA_MATEMATICA: int, LINGUA_ESTR: string, STATUS_REDACAO: string, NOTA_COMP1: int, NOTA_COMP2: int, NOTA_COMP3: int, NOTA_COMP4: int, NOTA_COMP5: int, NOTA_REDACAO: int]

Analisando os tipos das variáveis do arquivo, estão todas classificadas como string. As variáveis que não condizem com essa modelagem para análise receberão um novo formato.

In [25]:
list(enem_gold.schema)

[StructField('INSCRICAO', StringType(), True),
 StructField('ANO', StringType(), True),
 StructField('FAIXA_ETARIA', StringType(), True),
 StructField('SEXO', StringType(), True),
 StructField('TP_ESTADO_CIVIL', StringType(), True),
 StructField('COR_RACA', StringType(), True),
 StructField('TP_NACIONALIDADE', StringType(), True),
 StructField('ST_CONCLUSAO', StringType(), True),
 StructField('ANO_CONCLUIU', StringType(), True),
 StructField('ESCOLA', StringType(), True),
 StructField('ENSINO', StringType(), True),
 StructField('TREINEIRO', StringType(), True),
 StructField('CO_MUNICIPIO_ESC', StringType(), True),
 StructField('NO_MUNICIPIO_ESC', StringType(), True),
 StructField('CO_UF_ESC', StringType(), True),
 StructField('SG_UF_ESC', StringType(), True),
 StructField('TP_DEPENDENCIA_ADM_ESC', StringType(), True),
 StructField('TP_LOCALIZACAO_ESC', StringType(), True),
 StructField('TP_SIT_FUNC_ESC', StringType(), True),
 StructField('CO_MUNICIPIO_PROVA', StringType(), True),
 Stru

In [26]:
# Importando o pacote types do PySpark para alterar os dados do dataframe
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [27]:
# alterando o formato dos dados no df
enem_gold = enem_gold.select(
 enem_gold.INSCRICAO.cast(StringType()),
 enem_gold.ANO.cast(StringType()),
 enem_gold.FAIXA_ETARIA.cast(StringType()),
 enem_gold.SEXO.cast(StringType()),
 enem_gold.COR_RACA.cast(StringType()),
 enem_gold.ST_CONCLUSAO.cast(StringType()),
 enem_gold.ANO_CONCLUIU.cast(StringType()),
 enem_gold.ESCOLA.cast(StringType()),
 enem_gold.ENSINO.cast(StringType()),
 enem_gold.TREINEIRO.cast(StringType()),
 enem_gold.UF_PROVA.cast(StringType()),
 enem_gold.PRE_C_NATURAIS.cast(StringType()),
 enem_gold.PRE_C_HUMANAS.cast(StringType()),
 enem_gold.PRE_LINGUAGENS.cast(StringType()),
 enem_gold.PRE_MATEMATICA.cast(StringType()),
 enem_gold.NOTA_C_NATURAIS.cast(IntegerType()),
 enem_gold.NOTA_C_HUMANAS.cast(IntegerType()),
 enem_gold.NOTA_LINGAGENS.cast(IntegerType()),
 enem_gold.NOTA_MATEMATICA.cast(IntegerType()),
 enem_gold.LINGUA_ESTR.cast(StringType()),
 enem_gold.STATUS_REDACAO.cast(StringType()),
 enem_gold.NOTA_COMP1.cast(IntegerType()),
 enem_gold.NOTA_COMP2.cast(IntegerType()),
 enem_gold.NOTA_COMP3.cast(IntegerType()),
 enem_gold.NOTA_COMP4.cast(IntegerType()),
 enem_gold.NOTA_COMP5.cast(IntegerType()),
 enem_gold.NOTA_REDACAO.cast(IntegerType())
)

# visualizando o schema do df após alteração
list(enem_gold.schema)

[StructField('INSCRICAO', StringType(), True),
 StructField('ANO', StringType(), True),
 StructField('FAIXA_ETARIA', StringType(), True),
 StructField('SEXO', StringType(), True),
 StructField('COR_RACA', StringType(), True),
 StructField('ST_CONCLUSAO', StringType(), True),
 StructField('ANO_CONCLUIU', StringType(), True),
 StructField('ESCOLA', StringType(), True),
 StructField('ENSINO', StringType(), True),
 StructField('TREINEIRO', StringType(), True),
 StructField('UF_PROVA', StringType(), True),
 StructField('PRE_C_NATURAIS', StringType(), True),
 StructField('PRE_C_HUMANAS', StringType(), True),
 StructField('PRE_LINGUAGENS', StringType(), True),
 StructField('PRE_MATEMATICA', StringType(), True),
 StructField('NOTA_C_NATURAIS', IntegerType(), True),
 StructField('NOTA_C_HUMANAS', IntegerType(), True),
 StructField('NOTA_LINGAGENS', IntegerType(), True),
 StructField('NOTA_MATEMATICA', IntegerType(), True),
 StructField('LINGUA_ESTR', StringType(), True),
 StructField('STATUS_RE

In [33]:
enem_gold.limit(5)

DataFrame[INSCRICAO: string, ANO: string, FAIXA_ETARIA: string, SEXO: string, COR_RACA: string, ST_CONCLUSAO: string, ANO_CONCLUIU: string, ESCOLA: string, ENSINO: string, TREINEIRO: string, UF_PROVA: string, PRE_C_NATURAIS: string, PRE_C_HUMANAS: string, PRE_LINGUAGENS: string, PRE_MATEMATICA: string, NOTA_C_NATURAIS: int, NOTA_C_HUMANAS: int, NOTA_LINGAGENS: int, NOTA_MATEMATICA: int, LINGUA_ESTR: string, STATUS_REDACAO: string, NOTA_COMP1: int, NOTA_COMP2: int, NOTA_COMP3: int, NOTA_COMP4: int, NOTA_COMP5: int, NOTA_REDACAO: int]

Finalizando o arquivo agora com os dados tratados e selecionados, resta salvar no diretório do último estágio do processo de ETL: gold.

In [30]:
# Caminho para salvar o arquivo gold
gold_path = "/content/enem_2020/3.gold/ENEM_2020_GOLD.parquet"

# Salvar o DataFrame na pasta gold
enem_gold.write.parquet(gold_path)
#dados_enem.write.mode("overwrite").parquet(parquet_path)